# Spark DataFrames Project Exercise

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [0]:
df = sqlContext.sql('SELECT * FROM walmart_stock')
df.show(2)

+-------------------+-----+-----+-----+-----+--------+---------+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|
+-------------------+-----+-----+-----+-----+--------+---------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|52.619236|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|52.078476|
+-------------------+-----+-----+-----+-----+--------+---------+
only showing top 2 rows



#### What are the column names?

In [0]:
df.columns

Out[25]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### Print out the first 5 columns.

In [0]:
df.select(['Date', 'Open', 'High', 'Low', 'Close']).show(2)

+-------------------+-----+-----+-----+-----+
|               Date| Open| High|  Low|Close|
+-------------------+-----+-----+-----+-----+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71|
+-------------------+-----+-----+-----+-----+
only showing top 2 rows



#### Use describe() to learn about the DataFrame.

In [0]:
df.describe().show()

+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|             Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|             1258|             1258|             1258|             1258|             1258|             1258|
|   mean|72.35785375452572| 72.8393880756178|71.91860094964979|72.38844997363553|8222093.481717011|67.23883840200064|
| stddev|6.768090251767697|6.768186825250206|6.744075739203606|6.756859160119612|  4519780.8431556|6.722609385249684|
|    min|            56.39|            57.06|             56.3|            56.42|          2094900|         50.36369|
|    max|             90.8|            90.97|            89.25|            90.47|         80898100|        84.914215|
+-------+-----------------+-----------------+-----------

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [0]:
from pyspark.sql.functions import format_number

In [0]:
result = df.describe()
result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [0]:
df2 = df.withColumn("HV Ratio",df["High"]/df["Volume"])#.show()
# df2.show()
df2.select('HV Ratio').show(2)

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714682786927E-6|
|6.290848662516662E-6|
+--------------------+
only showing top 2 rows



#### What day had the Peak High in Price?

In [0]:
df.orderBy(df['High'].desc()).show(1)

+-------------------+----+-----+-----+-----+-------+---------+
|               Date|Open| High|  Low|Close| Volume|Adj Close|
+-------------------+----+-----+-----+-----+-------+---------+
|2015-01-13 00:00:00|90.8|90.97|88.93|89.31|8215400| 83.82545|
+-------------------+----+-----+-----+-----+-------+---------+
only showing top 1 row



#### What is the mean of the Close column?

In [0]:
from pyspark.sql.functions import mean
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844997363553|
+-----------------+



#### What is the max and min of the Volume column?

In [0]:
from pyspark.sql.functions import max, min

In [0]:
df.select(max('Volume'), min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [0]:
df.show(3)

+-------------------+-----+-----+-----+-----+--------+---------+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|
+-------------------+-----+-----+-----+-----+--------+---------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|52.619236|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|52.078476|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200| 51.82554|
+-------------------+-----+-----+-----+-----+--------+---------+
only showing top 3 rows



In [0]:
"""%sql
SELECT COUNT(1)
FROM walmart_stock
WHERE CLOSE < 60"""


df.filter('Close < 60').count()
df.filter(df['Close'] < 60).count()

Out[36]: 81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [0]:
(df.filter(df['High']>80).count()*1.0/df.count())*100

Out[37]: 9.141494435612083

#### What is the max High per year?

In [0]:
from pyspark.sql.functions import year

In [0]:
df1 = df.withColumn('Year', year(df['Date']))

In [0]:
df1.show(2)

+-------------------+-----+-----+-----+-----+--------+---------+----+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|Year|
+-------------------+-----+-----+-----+-----+--------+---------+----+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|52.619236|2012|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|52.078476|2012|
+-------------------+-----+-----+-----+-----+--------+---------+----+
only showing top 2 rows



In [0]:
df1.groupBy('Year')\
   .max('High')\
   .show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|    90.97|
|2013|    81.37|
|2014|    88.09|
|2012|     77.6|
|2016|    75.19|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months.

In [0]:
df1.show(2)

+-------------------+-----+-----+-----+-----+--------+---------+----+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|Year|
+-------------------+-----+-----+-----+-----+--------+---------+----+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|52.619236|2012|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|52.078476|2012|
+-------------------+-----+-----+-----+-----+--------+---------+----+
only showing top 2 rows



In [0]:
from pyspark.sql.functions import month, avg

In [0]:
df2 = df1.withColumn("Month", month(df["Date"]))
df2.show(2)

+-------------------+-----+-----+-----+-----+--------+---------+----+-----+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|Year|Month|
+-------------------+-----+-----+-----+-----+--------+---------+----+-----+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|52.619236|2012|    1|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|52.078476|2012|    1|
+-------------------+-----+-----+-----+-----+--------+---------+----+-----+
only showing top 2 rows



In [0]:
df2.groupBy('Month').agg(avg('Close')).orderBy('Month', ascending=True).show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1| 71.4480196131338|
|    2|71.30680438169499|
|    3|71.77794376266337|
|    4|72.97361900692894|
|    5|72.30971685445533|
|    6| 72.4953774506191|
|    7|74.43971944078106|
|    8| 73.0298185521906|
|    9|72.18411782208611|
|   10| 71.5785454489968|
|   11|72.11108927207418|
|   12|72.84792482628012|
+-----+-----------------+

